<a href="https://colab.research.google.com/github/ragyeongyoon/langchain-playground/blob/main/6_1_FewShot_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필수패키지 설치
- langchain
- https://python.langchain.com/docs/concepts/few_shot_prompting/

In [ ]:
!pip install -qU langchain

2. LLM 준비

In [ ]:
!pip install -qU langchain_openai
!pip install -qU langchain_google_genai

In [ ]:
import os
from google.colab import userdata

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

user_secret_name = 'KU_OPENAI_API_KEY'
api_key_value = userdata.get(user_secret_name)
os.environ['OPENAI_API_KEY'] = api_key_value

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

chat1 = ChatOpenAI(model='gpt-4.1-nano')
chat2 = ChatGoogleGenerativeAI(model='models/gemini-1.5-flash')

## 3. Generating Example
- https://python.langchain.com/docs/how_to/few_shot_examples/

이 섹션에서는 Few-Shot Prompting의 가장 기본적인 형태를 다룹니다. LLM에게 몇 가지(Few) 예시(Shot)를 먼저 보여주고, 비슷한 형식으로 답변을 생성하도록 유도하는 기법입니다.

**1) 간단한 질문**

아무런 예시 없이, LLM에게 직접 질문을 던집니다. 모델은 자신의 지식을 바탕으로 "코카콜라 제로"가 먼저 출시되었다고 간단히 답변합니다.

In [ ]:
question = '펩시제로와 코카콜라제로 중 먼저 출시된 것은?'
chat1.invoke(question)

**2) Few-Shot 예제 정의 및 프롬프트 템플릿 생성**

* examples: LLM에게 제공할 예시들입니다. 각 예시는 '질문(question)'과 '답변(answer)'으로 구성됩니다. 특히 '답변' 부분은 단순히 최종 답만 있는 것이 아니라, 정답을 찾기 위한 사고 과정(Chain of Thought), 즉 중간 질문과 답변을 포함하고 있습니다. 이것이 LLM이 따라 하길 바라는 패턴입니다.


In [ ]:
examples = [
    {
        'question':'펩시 제로와 코카콜라 제로 중 먼저 출시된 것은?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 펩시 제로는 언제 출시 되었나요?
                    중간답변: 2007년 입니다.
                    중간질문: 코카콜라 제로는 언제 출시 되었나요?
                    중간답변: 2005년 입니다.
                    최종답변: 코카콜라 제로'''
     },
    {
        'question':'코카콜라의 창업자는 언제 태어났나요?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 코카콜라의 창업자는 누구인가요?
                    중간답변: 존 스티스 펨버턴 입니다.
                    중간질문: 존 스티스 펨버턴은 언제 태어났나요?
                    중간답변: 1831년 7월 8일 입니다.
                    최종답변: 1831년'''
     },
    {
        'question':'펩시와 코카콜라의 청업자는 같은 도시에서 태어났나요?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 펩시의 창업자는 누구인가요?
                    중간답변: 케일럽 브래덤 입니다.
                    중간질문: 케일럽 브래덤은 어디에서 태어났나요?
                    중간답변: 미국 노스캐롤라이나주 입니다.
                    중간질문: 코카콜라의 창업자는 누구인가요?
                    중간답변: 존 스티스 펨버턴 입니다.
                    중간질문: 존 스티스 펨버턴은 어디에서 태어났나요?
                    중간답변: 미국 조지아주 입니다.
                    최종답변: 아니오'''
     }
]

* PromptTemplate: examples 리스트에 있는 각 딕셔너리를 LLM이 이해할 수 있는 문자열 형식으로 만들어주는 틀입니다. {question}과 {answer} 부분에 딕셔너리의 값이 채워집니다.

In [ ]:
example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")
print(example_prompt.invoke(examples[0]).to_string())

**3) FewShotPromptTemplate 활용 및 실행**

* FewShotPromptTemplate: Few-Shot Prompting을 위해 설계된 핵심 클래스입니다.

* examples: 위에서 정의한 예제 목록입니다
* example_prompt: 각 예제를 어떤 형식으로 만들지 정의한 PromptTemplate입니다.
* suffix: 모든 예제가 나열된 후, 마지막에 사용자의 실제 질문이 들어갈 부분을 정의합니다. {input} 변수에 실제 질문이 담깁니다.
* input_variables: 사용자의 입력이 담길 변수 이름(input)을 명시합니다.

동작 방식:

prompt_template.invoke(...)가 호출되면, FewShotPromptTemplate은 examples 목록의 모든 예제를 example_prompt 형식에 맞춰 차례대로 문자열로 만듭니다.

그 뒤에 suffix에 정의된 대로 실제 질문("질문: 아이폰13과...")을 덧붙입니다.

이렇게 완성된 전체 프롬프트를 LLM(chat1)에게 전달합니다.

> LLM은 앞서 제공된 "펩시 vs 콜라" 예제의 사고 과정을 그대로 모방하여, '아이폰13 출시일'과 '갤럭시S22 출시일'을 순서대로 질문하고 답하며 최종 결론을 내립니다. 이는 복잡한 질문을 단계별로 분해하여 푸는 능력을 유도한 것입니다.

In [ ]:
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)

print(chat1.invoke(prompt_template.invoke('아이폰13과 갤럭시S22 중 먼저 출시된 것은?')).content)
# print(chat1.invoke(prompt_template.invoke('삼성전자의 창업주는 언제 태어났나요?')).content)
# print(chat1.invoke(prompt_template.invoke('삼성과 LG의 창업주는 같은 도시에서 태어났나요?')).content)

In [ ]:
examples = [
    {
        'question':'대한민국에서 프랑스 파리까지 24시간 안에 다녀올 수 있나요?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 파리와 대한민국의 거리는 얼마인가요?
                    중간답변: 약 9,000km입니다.
                    중간질문: 비행기를 이용하여 대한민국에서 파리로 갈 때, 대략 몇시간 걸리나요?
                    중간답변: 9000/(900-180)이므로 약 12.5시간 입니다.
                    중간질문: 비행기를 이용하여 파리에서 대한민국으로 올 때, 대략 몇시간 걸리나요?
                    중간답변: 12.5-2이므로 약 10시간 입니다.
                    중간질문: 비행기를 이용하여 대한민국에서 파리 왕복 비행 시간은 대략 몇시간 인가요?
                    중간답변: 12.5+10이므로 약 22.5시간 입니다.
                    중간질문: 공항 체류시간은 평균 몇 시간인가요?
                    중간답변: 약 2시간 입니다.
                    중간질문: 비행기 왕복 시간과 공항 체류시간을 고려할 때 전체 몇시간 걸리나요?
                    중간답변: 약 24.5시간 입니다.
                    최종답변: 불가능합니다.
     '''
     }
]

prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)

print(chat1.invoke(prompt_template.invoke('대한민국에서 미국 하와이까지 24시간 안에 다녀올 수 있나요?')).content)

In [ ]:
examples = [
    {
        'question':'아이스아메리카노 5천원, 점심식대 1만원, 친목회 회비 미납금 1만원, 간식비용 1만원(친구가 사줌)일때, 오늘 총 사용금액은?',
        'answer':'''
            이 질문에 추가질문이 필요한가요? 네.
            중간질문: 내가 지출한 항목은 무엇인가요?
            중간답변: 아이스아메리카노, 점심식대
            중간질문: 내가 지출하지 않은 항목은 무엇인가요?
            중간답변: 친목회 회비, 간식비용
            중간질문: 지출한 항목의 총합은 얼마인가요?
            중간답변: 5천원 + 1만원 = 1.5만원
            최종답변: 만오천원
     '''
     }
]

prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)

print(chat1.invoke(prompt_template.invoke('톨게이트 비용 5천원, 주유비 3만원(주유비는 공금에서 차감), 뻥튀기 만원일때, 오늘 총 사용금액은?')).content)

## 4. ExampleSelector
- https://python.langchain.com/docs/how_to/few_shot_examples/#using-an-example-selector
- https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html

Few-Shot Prompting에서 모든 예제를 항상 제공하는 것은 비효율적일 수 있습니다. 프롬프트가 너무 길어지면 비용이 증가하고 모델의 컨텍스트 길이 제한을 초과할 수 있습니다. ExampleSelector는 사용자의 질문과 가장 관련 높은 예제만 동적으로 선택하여 프롬프트에 포함시키는 역할을 합니다.

In [ ]:
!pip install -qU langchain_chroma

In [ ]:
examples = [
    {
        'question':'펩시 제로와 코카콜라 제로 중 먼저 출시된 것은?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 펩시 제로는 언제 출시 되었나요?
                    중간답변: 2007년 입니다.
                    중간질문: 코카콜라 제로는 언제 출시 되었나요?
                    중간답변: 2005년 입니다.
                    최종답변: 코카콜라 제로'''
     },
    {
        'question':'코카콜라의 창업자는 언제 태어났나요?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 코카콜라의 창업자는 누구인가요?
                    중간답변: 존 스티스 펨버턴 입니다.
                    중간질문: 존 스티스 펨버턴은 언제 태어났나요?
                    중간답변: 1831년 7월 8일 입니다.
                    최종답변: 1831년'''
     },
    {
        'question':'펩시와 코카콜라의 청업자는 같은 도시에서 태어났나요?',
        'answer':'''이 질문에 추가질문이 필요한가요? 네.
                    중간질문: 펩시의 창업자는 누구인가요?
                    중간답변: 케일럽 브래덤 입니다.
                    중간질문: 케일럽 브래덤은 어디에서 태어났나요?
                    중간답변: 미국 노스캐롤라이나주 입니다.
                    중간질문: 코카콜라의 창업자는 누구인가요?
                    중간답변: 존 스티스 펨버턴 입니다.
                    중간질문: 존 스티스 펨버턴은 어디에서 태어났나요?
                    중간답변: 미국 조지아주 입니다.
                    최종답변: 아니오'''
     }
]

example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="질문: {input}",
    input_variables=["input"],
)
print(prompt_template.invoke('아이폰13과 세종대왕 중 시기가 먼저인 것은?').to_messages()[0].content)

**SemanticSimilarityExampleSelector 설정**

1. from_examples가 실행될 때, examples 목록의 모든 'question'들이
OpenAIEmbeddings를 통해 숫자 벡터로 변환되어 Chroma DB에 저장됩니다.

  * OpenAIEmbeddings: 텍스트를 임베딩(Embedding), 즉 의미를 압축한 숫자 벡터(vector)로 변환하는 모델입니다. 단어의 의미, 문맥을 파악하여 벡터를 생성합니다.

2. example_selector.select_examples()가 호출되면, 새로운 질문('테슬라와 청동거울...') 역시 벡터로 변환됩니다.

3. Chroma DB는 이 새로운 질문 벡터와 가장 가까운(유사한) 기존 예제 벡터를 찾아 반환합니다.

  * Chroma: 임베딩된 벡터들을 저장하고, 특정 벡터와 가장 유사한 다른 벡터들을 효율적으로 찾아주는 **벡터 스토어(Vector Store)**)


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(model='text-embedding-3-small'),
    Chroma,
    k=1,
)

question = '테슬라와 청동거울 중 시기가 먼저인 것은?'
selected_examples = example_selector.select_examples({'question': question})
print(selected_examples[0]['question'])
print(selected_examples[0]['answer'])

**ExampleSelector를 사용한 FewShotPromptTemplate**

FewShotPromptTemplate을 만들 때 examples=... 대신 example_selector=...를 사용합니다.

1. prompt.invoke(...)가 호출되면, FewShotPromptTemplate은 example_selector에게 현재 입력('훈민정음과...')을 전달합니다.

2. example_selector는 의미적으로 가장 유사한 예제(이 경우 '코카콜라 창업자' 예제)를 k=1개 선택하여 반환합니다.

3. FewShotPromptTemplate은 선택된 그 예제만을 프롬프트에 포함시켜 LLM에게 전달합니다.

> LLM은 "창업자의 출생 연도"를 묻는 예제 형식을 참고하여, '훈민정음 창제 연도'와 '일론 머스크 아버지 출생 연도'를 각각 묻고 비교하는 방식으로 답변을 생성합니다. 이는 질문의 의도에 맞는 예제를 동적으로 활용하여 더 정확한 답변을 유도한 것입니다.




In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix='질문: {input}',
    input_variables=['input'],
)

print(prompt.invoke('훈민정음과 일론 머스크의 아버지 중 시기가 먼저인 것은?').to_messages()[0].content)

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix='질문: {input}',
    input_variables=['input'],
)

chat1.invoke(prompt.invoke('훈민정음과 일론 머스크의 아버지 중 시기가 먼저인 것은?'))

5. FewShotExamples
- https://python.langchain.com/docs/how_to/few_shot_examples_chat/
- https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html#langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate
- https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html

**FewShotExamples (ChatModel용)**

이전까지는 단일 텍스트로 프롬프트를 구성했지만, ChatModel은 System, Human, AI와 같은 역할이 부여된 메시지 목록을 입력으로 받는 것이 더 자연스럽습니다. 이 섹션은 채팅 형식에 맞는 Few-Shot Prompting 방법을 다룹니다.

* ChatPromptTemplate: ('역할', '내용') 튜플의 리스트를 받아 채팅 메시지 형식을 만듭니다.
* FewShotChatMessagePromptTemplate: 채팅 모델을 위한 Few-shot 프롬프트 템플릿입니다. examples에 있는 각 딕셔너리를 example_prompt 형식(Human 메시지, AI 메시지)에 맞춰 변환합니다.

1) LLM에게 보여줄 몇 가지(Few-shot) 대화 예제를 정의합니다.

  예제는 user(사용자 질문)와 ai(AI의 답변)로 구성된 파이썬 딕셔너리입니다. 여기서는 간단한 제곱 계산을 예시로 들었습니다.

In [ ]:
examples = [
    {'user':'2 ^ 2', 'ai':'4'},
    {'user':'3 ^ 2', 'ai':'9'},
    {'user':'4 ^ 2', 'ai':'16'}
]

**2) 채팅용 Few-Shot 프롬프트 생성**

  앞서 정의한 examples 데이터를 실제 채팅 메시지 형식([HumanMessage, AIMessage, ...])으로 변환할 준비를 합니다.

  example_prompt: ChatPromptTemplate을 사용해 단 1개의 예제가 어떤 모양일지 정의합니다. {user} 키의 값은 user 역할 메시지로, {ai} 키의 값은 ai 역할 메시지로 만듭니다.

* few_shot_prompt: FewShotChatMessagePromptTemplate의 핵심 기능입니다.

* example_prompt: 각 예제를 포맷할 때 사용할 템플릿입니다.

* examples: 포맷할 실제 데이터 목록입니다.
이 코드는 examples 목록에 있는 모든 딕셔너리를 example_prompt 형식에 맞춰 변환할 수 있는 객체를 생성합니다.

* print(...): few_shot_prompt가 실제로 examples 데이터를 [HumanMessage, AIMessage, ...] 순서의 리스트로 잘 변환하는지 확인하는 과정입니다.

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ('user', '{user}'),
    ('ai', '{ai}')
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)


print(few_shot_prompt.invoke({}).to_messages())

**최종 프롬프트 조합 및 LLM 호출**

시스템 역할, Few-shot 예제, 그리고 실제 사용자 질문을 모두 합쳐 완전한 프롬프트를 만든 뒤 LLM에게 전달하여 답변을 생성합니다.

* final_prompt: ChatPromptTemplate으로 전체 대화의 구조를 설계합니다.

* ('system', ...): 모델에게 '수식마법사'라는 역할을 부여합니다.

* few_shot_prompt: 이 자리에 위에서 만든 예제(2^2=4, 3^2=9 등)들이 모두 들어갑니다.

* ('user', '{input}'): 마지막에 사용자의 실제 질문(2 ^ 10)이 들어갑니다.

* chat1.invoke(...): 생성된 최종 프롬프트를 chat1 모델에 전달합니다. 모델은 '수식마법사' 역할과 제곱 계산 예제들을 보고, 사용자의 질문 2 ^ 10에 대한 답 1024를 생성합니다.

In [ ]:
final_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 수식마법사 입니다.'),
    few_shot_prompt,
    ('user', '{input}')
])

chat1.invoke(final_prompt.invoke('2 ^ 10'))

**동적 예제 선택(ExampleSelector)을 사용한 Few-Shot 채팅 프롬프트**

**[In 25] 임베딩 모델 준비**

텍스트를 **의미론적 벡터(숫자의 배열)**로 변환하는 도구를 준비합니다.

* OpenAIEmbeddings는 문장의 의미를 컴퓨터가 이해할 수 있는 숫자 벡터로 만드는 역할을 합니다. '고양이'와 '강아지'는 '자동차'보다 벡터 공간에서 더 가깝게 위치하게 됩니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

**[In 26] 벡터 스토어 생성**

* data = ...: examples의 각 딕셔너리를 "2 ^ 2 = 4" 와 같은 하나의 문자열로 만듭니다.

* Chroma.from_texts(...): Chroma라는 벡터 스토어를 생성합니다. data의 텍스트들을 embeddings 모델로 벡터화하여 저장하고, 원본 예제 데이터(examples)도 함께 보관합니다.

In [ ]:
from langchain_chroma import Chroma

data = [' = '.join(example.values()) for example in examples]

vector_store = Chroma.from_texts(data, embeddings, examples)

**[In 27] ~ [In 29] 벡터 스토어 테스트 및 확장**

[In 27]: vector_store.similarity_search(...)를 통해 '2 ^ 10'과 가장 유사한 예제가 '2 ^ 2 = 4'임을 확인합니다.

[In 28]: vector_store.add_texts(...)를 사용하여 기존에 없던 "고양이에 대한 속담" 예제를 벡터 스토어에 동적으로 추가합니다.

[In 29]: 이제 "강아지 속담"을 검색하면, 의미적으로 가장 유사한 "고양이 속담" 예제를 찾아내는 것을 보여줍니다. 벡터 스토어가 확장되었음을 의미합니다.

In [ ]:
vector_store.similarity_search('2 ^ 10', k=1)

In [ ]:
vector_store.add_texts(['고양이에 대한 속담 말해줘 : 얌전한 고양이 부뚜막에 먼저 올라간다'], [{'user':'고양이에 대한 속담 말해줘','ai':'얌전한 고양이 부뚜막에 먼저 올라간다'}])

In [ ]:
vector_store.similarity_search('강아지에 대한 속담 말해줘', k=1)

[In 30] 의미 기반 예제 선택기 생성

사용자의 질문과 의미적으로 가장 유사한 예제를 벡터 스토어에서 자동으로 찾아주는 선택기를 만듭니다.

* vectorstore=vector_store: 검색할 대상이 되는 벡터 스토어를 지정합니다.

* k=1: 가장 유사한 예제를 1개만 선택하라고 설정합니다.

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=1
)

In [ ]:
example_selector.select_examples({'input':'2 ^ 20'}), \
example_selector.select_examples({'input':'강아지에 대한 속담 말해줘'})

**예제 선택기를 포함한 최종 프롬프트 생성**

사용자의 질문 종류에 따라 가장 적절한 예제 하나만을 동적으로 프롬프트에 포함시키는 최종 프롬프트를 구성합니다.

* few_shot_prompt: 이전과 달리 고정된 examples 리스트 대신 example_selector를 사용합니다. 이제 이 템플릿은 사용자의 입력(input)이 들어오면 example_selector를 호출하여 가장 유사한 예제를 가져옵니다.

* final_prompt: 전체 구조는 동일하지만, few_shot_prompt 부분이 이제 동적으로 변하는 부분이 되었습니다.

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [('user', '{user}'), ('ai', '{ai}')]
    ),
    input_variables=['input']
)

few_shot_prompt.invoke('A ? B').to_messages()[0]

최종 채팅 프롬프트 구성 및 실행
final_prompt: 전체 대화의 구조를 정의합니다.

* ('system', ...): 모델에게 역할을 부여합니다. (역할: 수식마법사)

* few_shot_prompt: examples에 있던 2^2=4, 3^2=9 등의 대화 예시들이 여기에 삽입됩니다.

* ('user', '{input}'): 사용자의 실제 질문이 마지막에 Human 메시지로 들어갑니다.

> LLM은 '수식마법사'라는 역할을 인지하고, 앞에 제시된 제곱 계산 예시들을 참고하여 2 ^ 10에 대한 답 1024를 정확히 계산해냅니다.

In [ ]:
final_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 예제를 보고 사용자 입력에 적절히 대답하는 시스템 입니다.'),
    few_shot_prompt,
    ('user', '{input}')
])

final_prompt.invoke('2 ^ 10'), \
final_prompt.invoke('강아지에 대한 속담 말해줘')

[In 34] 최종 LLM 호출 및 결과

기능: 두 가지 다른 종류의 질문으로 최종 프롬프트를 호출하여, 각 질문에 맞는 예제가 동적으로 선택되어 LLM에 전달되는 것을 확인합니다.


'A ^ 10'을 입력하면: example_selector가 제곱 계산 예제를 선택합니다. LLM은 그 예제를 보고 계산과 관련된 답변을 합니다.

'강아지에 대한 속담 말해줘'를 입력하면: example_selector가 속담 예제를 선택합니다. LLM은 속담 예제를 보고, 그에 맞는 새로운 속담을 생성해줍니다.

In [ ]:
chat1.invoke(final_prompt.invoke('A ^ 10')), \
chat1.invoke(final_prompt.invoke('강아지에 대한 속담 말해줘'))